In [1]:
import re
import json
import requests
import os
import time

In [6]:
# ../datas/item_pages/の一覧を取得する
directory_path = '../datas/html_receipes/item_pages/'

files = os.listdir(directory_path)

In [27]:
def parse_recipe_json(file_path):
    file_path = f'{directory_path}/{file_path}'  # HTMLファイルのパスを指定

    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # <script>タグ内のJSONデータを正規表現で抽出
    pattern = r'<script id="allrecipes-schema_1-0"[^>]*type="application/ld\+json"[^>]*>(.*?)</script>'
    match = re.search(pattern, html_content, re.DOTALL)
    json_content = match.group(1).strip()
    json_data = json.loads(json_content)
    print(json_data[0])
    for item in json_data[0]:
        print("-----")
        print(item["recipeIngredient"])
        #print(json_content)
        #print(json_content["recipeCategory"])
        #print(json_content["recipeCuisine"])
        #print(json_content["recipeIngredient"])
        #print(json_content["recipeInstructions"])
    

In [28]:
for file_path in files:
    parse_recipe_json(file_path)
    break

{'@context': 'http://schema.org', '@type': ['Recipe'], 'headline': 'Greek Beef Stuffed Onions (Salantourmasi)', 'datePublished': '2024-08-27T09:00:00.000-04:00', 'dateModified': '2024-09-05T17:40:56.577-04:00', 'author': [{'@type': 'Person', 'name': 'John Mitzewich', 'url': 'https://www.allrecipes.com/author/chef-john/'}], 'description': 'Chef John makes Greek beef stuffed onions—salantourmasi—for the first time, and the results are delicious. Onions are separated into layers, filled with Greek-seasoned beef and rice, and then slow-baked in a tomato-onion broth.', 'image': {'@type': 'ImageObject', 'url': 'https://www.allrecipes.com/thmb/VP2_i0liUEXF8TlKAfzUIDGaemw=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/8699944_Greek-Beef-Stuffed-Onions_John-Mitzewich_4x3.2-40f079e6e344419c8d273057ef50c496.jpg', 'height': 1125, 'width': 1500}, 'video': {'@type': 'VideoObject', 'contentUrl': 'https://content.jwplatform.com/videos/e7FhPeAl-K3AjnAEN.mp4', 'description': 'In this video, 

TypeError: string indices must be integers